<a href="https://colab.research.google.com/github/arjasc5231/moodots/blob/ACRNN/SER/CNN/CRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
learning_rate = 0.001
training_epochs = 60
batch_size = 100

In [3]:
mel_X_train, mel_X_test, mel_Y_train, mel_Y_test = np.load("/content/drive/MyDrive/team_runner/colab/dataset/emoDB/emo_mel_more.npy", allow_pickle=True)

print(mel_X_train.shape)
print(mel_Y_train.shape)
print(mel_X_test.shape)
print(mel_Y_test.shape)

(683, 128, 128)
(683,)
(228, 128, 128)
(228,)


In [4]:
mel_X_train = np.expand_dims(mel_X_train, axis=-1)
mel_X_test = np.expand_dims(mel_X_test, axis=-1)

mel_Y_train = to_categorical(mel_Y_train, 7)
mel_Y_test = to_categorical(mel_Y_test, 7)    

mel_train_dataset = tf.data.Dataset.from_tensor_slices((mel_X_train, mel_Y_train)).batch(batch_size)
mel_test_dataset = tf.data.Dataset.from_tensor_slices((mel_X_test, mel_Y_test)).batch(batch_size)

In [38]:
# 일단 매우매우 얕게 만들었음
def create_model():
    inputs = keras.Input(shape=(128, 128, 1))
    conv1 = keras.layers.Conv2D(filters=32, kernel_size=[3, 3], activation=tf.nn.relu)(inputs)
    pool1 = keras.layers.MaxPool2D(padding='SAME')(conv1)
    conv2 = keras.layers.Conv2D(filters=64, kernel_size=[3, 3], activation=tf.nn.relu)(pool1)
    pool2 = keras.layers.MaxPool2D(padding='SAME')(conv2)
    conv3 = keras.layers.Conv2D(filters=128, kernel_size=[3, 3], activation=tf.nn.relu)(pool2)
    pool3 = keras.layers.MaxPool2D(padding='SAME')(conv3)

    trans = keras.layers.Permute((2,1,3))(pool3)
    reshape = keras.layers.Reshape((-1, 15*128))(trans) # 열 개수(freq축)*ch
    lstm = keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=False, dropout=0.5))(reshape)  #dropout을 바꿔도 큰 영향은 없는듯. 대부분 65%
    
    fc = keras.layers.Dense(7)(lstm)
    return keras.Model(inputs=inputs, outputs=fc)

In [40]:
model = create_model()
model.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 128, 128, 1)]     0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 126, 126, 32)      320       
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 61, 61, 64)        18496     
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 29, 29, 128)       73856     
_________________________________________________________________
max_pooling2d_38 (MaxPooling (None, 15, 15, 128)       0   

In [27]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, from_logits=True))    
    return loss

In [28]:
def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [29]:
def evaluate(model, images, labels):
    logits = model(images, training=False)
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

In [30]:
optimizer = tf.optimizers.Adam(learning_rate=learning_rate)

In [41]:
print('Learning started. It takes sometime.')
print('Epoch\tloss\t\ttrain acc\ttest acc')
for epoch in range(training_epochs):
    loss = 0.
    train_acc = 0.
    test_acc = 0.
    
    for images, labels in mel_train_dataset:
        grads = grad(model, images, labels)                
        optimizer.apply_gradients(zip(grads, model.variables)) 
        loss += loss_fn(model, images, labels)
        train_acc += evaluate(model, images, labels)
    loss = loss / len(mel_train_dataset)
    train_acc = train_acc / len(mel_train_dataset)
    
    for images, labels in mel_test_dataset:
        test_acc += evaluate(model, images, labels)
    test_acc = test_acc / len(mel_test_dataset)  

    print('{}  \t{:.4f}  \t{:.4f}  \t{:.4f}'.format(epoch+1, loss, train_acc, test_acc))

print('Learning Finished!')
#5분정도 소요

Learning started. It takes sometime.
Epoch	loss		train acc	test acc
1  	1.9219  	0.2233  	0.2200
2  	1.8968  	0.2390  	0.2200
3  	1.8497  	0.2987  	0.2652
4  	1.8890  	0.2361  	0.2200
5  	1.7852  	0.2599  	0.2981
6  	1.6515  	0.3671  	0.3652
7  	1.4478  	0.4512  	0.3800
8  	1.3053  	0.4691  	0.4395
9  	1.2054  	0.5290  	0.4967
10  	1.1388  	0.5509  	0.4900
11  	1.0952  	0.5798  	0.4867
12  	1.0416  	0.5986  	0.5200
13  	0.9493  	0.6335  	0.4910
14  	0.9079  	0.6307  	0.5990
15  	0.8748  	0.6692  	0.5600
16  	0.8160  	0.7041  	0.5633
17  	0.7553  	0.7141  	0.5519
18  	0.7815  	0.7158  	0.4781
19  	0.7171  	0.7422  	0.6276
20  	0.6364  	0.7710  	0.6019
21  	0.5879  	0.7948  	0.5648
22  	0.5770  	0.7976  	0.6124
23  	0.5373  	0.8245  	0.6005
24  	0.4798  	0.8517  	0.5733
25  	0.5205  	0.8231  	0.5719
26  	0.5240  	0.8265  	0.6243
27  	0.4748  	0.8505  	0.5633
28  	0.4251  	0.8600  	0.5481
29  	0.3839  	0.8880  	0.5481
30  	0.4376  	0.8722  	0.5581
31  	0.4046  	0.8940  	0.5552
32  	0.3583

In [ ]:
conf_mat = [[0]*7 for i in range(7)] #mat[real_label]=predicted_label list

for images, labels in mel_test_dataset:
  logits = model(images, training=False)
  logits_max = tf.math.argmax(logits,1)
  labels_max = tf.math.argmax(labels,1)
  for i in range(len(logits)): conf_mat[labels_max[i]][logits_max[i]]+=1

for i in range(7): print(conf_mat[i])

[46, 0, 0, 0, 2, 0, 0]
[0, 24, 2, 0, 1, 5, 4]
[5, 0, 18, 1, 0, 1, 4]
[7, 2, 0, 7, 3, 2, 5]
[7, 0, 7, 1, 6, 0, 3]
[0, 4, 1, 1, 0, 37, 0]
[1, 4, 1, 0, 0, 0, 16]


In [ ]:
conf_mat_normal = []
for i in range(7):
  s = sum(conf_mat[i])
  conf_mat_normal.append(list(map(lambda x:(x/s)*100, conf_mat[i])))

label = ['anger','boredom','disgust','fear','happy','sad','neutral']
print('\t'+'\t'.join(label))
for i in range(7):
  print(label[i], end='')
  for j in range(7): print('\t%2.0f'%conf_mat_normal[i][j], end=' ')
  print()

	anger	boredom	disgust	fear	happy	sad	neutral
anger	96 	 0 	 0 	 0 	 4 	 0 	 0 
boredom	 0 	67 	 6 	 0 	 3 	14 	11 
disgust	17 	 0 	62 	 3 	 0 	 3 	14 
fear	27 	 8 	 0 	27 	12 	 8 	19 
happy	29 	 0 	29 	 4 	25 	 0 	12 
sad	 0 	 9 	 2 	 2 	 0 	86 	 0 
neutral	 5 	18 	 5 	 0 	 0 	 0 	73 
